In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Data path
import pathlib
DATA_FOLDER = pathlib.Path("/home/jovyan/work/Dan/data")

import sys
sys.path.append('/home/jovyan/work/Dan/upload/src/')

# Math stuff
import numpy as np
from numpy import ndarray

# Data visualisation
import pandas as pd
from pandas import DataFrame

# Vectorize algotrithm
from gensim.models import Word2Vec

# Multidimensional space visualizer
from sklearn.manifold import TSNE

# Graphic visualization libraries
# Bokeh --
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

# Plotly
import plotly.express as px


In [2]:
# Open pickle file and read data
PATH = DATA_FOLDER / 'pwdb/pickle/df_columns_labels.pkl'
df = pd.read_pickle(PATH)
# Set which data we want to use to train word2vec model
columns = df['Concatenated Data (clean)']
columns

0      [hardship, case, fund, safety, net, selfemploy...
1      [state, support, tourism, access, financeas, t...
2      [bank, guarantees, smes, oneperson, enterprise...
3      [emergency, measures, relating, shorttime, wor...
4      [airbus, agreement, making, unworked, hours, p...
                             ...                        
930    [taxation, measures, enterprisesas, consequenc...
931    [winwinloanthe, winwin, loan, system, encourag...
932    [support, measures, businesses, closed, since,...
933    [new, loans, available, enterprises, brusselst...
934    [financial, support, teleworkers, increase, te...
Name: Concatenated Data (clean), Length: 935, dtype: object

In [3]:
# Insert our data and set minimal word count to 10, and size of each word to 300 vectors
dfVec = Word2Vec(columns, window=5, min_count=10, size=300)
w2v_dict = {w: vec for w, vec in zip(dfVec.wv.index2word, dfVec.wv.syn0)}
w2v = dfVec.wv.syn0

In [4]:
# Test the result
dfVec.most_similar('covid')


[('pandemic', 0.9931578040122986),
 ('crisis', 0.9928868412971497),
 ('measures', 0.9882986545562744),
 ('economic', 0.9727696776390076),
 ('affected', 0.9532006978988647),
 ('government', 0.9519243836402893),
 ('health', 0.947557806968689),
 ('emergency', 0.9286361336708069),
 ('spread', 0.9258859753608704),
 ('coronavirus', 0.918663740158081)]

In [5]:
dfVec.wv.most_similar_cosmul(positive=['covid', 'health'], negative=['economic'])

[('crisis', 0.9850180149078369),
 ('measures', 0.983648419380188),
 ('pandemic', 0.9828519821166992),
 ('affected', 0.9613243937492371),
 ('government', 0.9601290822029114),
 ('public', 0.9562671780586243),
 ('sector', 0.9558254480361938),
 ('spread', 0.9493266344070435),
 ('act', 0.9464930295944214),
 ('emergency', 0.9463182687759399)]

In [6]:
print(dfVec.wv.doesnt_match("covid costs legal".split()))

covid


In [14]:
# Save our trained model
# dfVec.save('/home/jovyan/work/Dan/data/pwdb/word2vec/df.model')
